In [ ]:
from fastai.vision.all import *
from pathlib import Path
import pandas as pd
import numpy as np
import plotly.graph_objs as go

In [ ]:
path = Path('../input/plant-seedlings-classification/')
train = path / 'train'
test = path / 'test'

In [ ]:
d = {}
for folder in train.ls():
    d[folder.name] = len(folder.ls())

s = pd.Series(d, name='No of images').reset_index()

In [ ]:
fig = go.Figure([go.Bar(
    x=s['index'], 
    y=s['No of images'], 
    text=s['No of images'])])
fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(title = 'No. of images per plant species')
fig.show()

In [ ]:
db = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   splitter=RandomSplitter(seed=0),
                   get_items=get_image_files, 
                   get_y=parent_label,
                item_tfms = RandomResizedCrop(256, min_scale=0.35))

In [ ]:
dl = db.dataloaders(train)

In [ ]:
dl.show_batch(nrows=3, ncols=3)

In [ ]:
model = cnn_learner(dl, resnet34, metrics=error_rate)
model.fine_tune(1, base_lr=0.1)

In [ ]:
model.lr_find()

In [ ]:
model = cnn_learner(dl, resnet34, metrics=error_rate)
model.fine_tune(1, base_lr=3e-3)

In [ ]:
model = cnn_learner(dl, resnet34, metrics=error_rate)
model.fit_one_cycle(1, 3e-3)
model.unfreeze()
model.fit_one_cycle(10, lr_max=slice(1e-6,1e-4))

In [ ]:
model.recorder.plot_loss()

In [ ]:
model.show_results()

In [ ]:
interp = ClassificationInterpretation.from_learner(model)

In [ ]:
interp.print_classification_report()

In [ ]:
interp.plot_confusion_matrix(figsize=(9,7), dpi=60)

In [ ]:
test_images = get_image_files(test)

In [ ]:
test_loader = dl.test_dl(test_images)

In [ ]:
preds = model.get_preds(dl=test_loader)

In [ ]:
predictions = torch.argmax(preds[0], dim=1)

In [ ]:
vocab = {key: val for key, val in enumerate(dl.vocab)}
vocab

In [ ]:
predicted_labels = []
for i in predictions:
    predicted_labels.append(vocab[i.float().item()])

In [ ]:
test_files = []
for i in test_images:
    test_files.append(i.name)

In [ ]:
submission_df = pd.DataFrame({'file': test_files,'species': predicted_labels})
submission_df.to_csv('submission.csv',index=False)